In [15]:
# !pip install gtts pydub audioop-lts
from gtts import gTTS
from pydub import AudioSegment
import time
import os
import pandas as pd
import glob
from collections import defaultdict
pd.options.display.max_columns = 100

# 0. Choose setting

In [31]:
recording_id, recording_name = '001', '0817_word_p2_k2_sent'
# recording_id, recording_name = '002', '0817_word_p2_k2_csent'
# recording_id, recording_name = '004', '0817_word_p2_k2_wordonly'
# recording_id, recording_name = '005', '0815_word_p2_k2_word_pinyin'
# recording_id, recording_name = '006', '0815_word_p2_k2_ecombo'

# 1. Load data

In [30]:
cols_keep = [
    'chinese', 'pinyin', 'english',
    'type', 'priority', 'known', 'known_pinyin_prompt', 'known_english_prompt',
    'phonetic', 'category1', 'quality',
    'word1', 'word1_english', 'word2', 'word2_english', 'word3', 'word3_english', 'word4', 'word4_english',
    'sentence', 'sentence_pinyin', 'sentence_english', 'date']
sheet_url = 'https://docs.google.com/spreadsheets/d/1pw9EAIvtiWenPDBFBIf7pwTh0FvIbIR0c3mY5gJwlDk/edit#gid=0'
sheet_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(sheet_url)[cols_keep]
df = df.dropna(subset=['chinese', 'english'])
df['known_english_prompt'] = df['known_english_prompt'].fillna(6)
print(df.shape)
df.head()

(4594, 23)


,chinese,pinyin,english,type,priority,known,known_pinyin_prompt,known_english_prompt,phonetic,category1,quality,word1,word1_english,word2,word2_english,word3,word3_english,word4,word4_english,sentence,sentence_pinyin,sentence_english,date
0,后备箱,hòu bèi xiāng,trunk;boot (of car),combo,1.0,5.0,1.0,3.0,NaN,travel,1.0,后,back,设备,equipment,箱,box,NaN,NaN,把行李放进后备箱,Bǎ xínglǐ fàng jìn hòubèixiāng,Put the luggage in the trunk,NaN
1,爆竹,bào zhú,firecracker,combo,1.0,3.0,2.0,5.0,NaN,china,1.0,爆,explode,竹,bamboo,NaN,NaN,NaN,NaN,春节的时候我们放了很多爆竹。,Chūnjié de shíhou wǒmen fàng le hěn duō bàozhú.,We set off many firecrackers during Spring Fes...,NaN
2,周一,zhōu yī,monday,combo,1.0,1.0,1.0,1.0,NaN,general,1.0,周,week,一,one,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,一月,yī yuè,january,combo,1.0,1.0,1.0,1.0,NaN,general,1.0,一,one,月,month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,黄金,huáng jīn,gold,combo,1.0,1.0,1.0,1.0,NaN,industry,1.0,黄,yellow,金,metal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Select data

In [ ]:
def filter_by_recording_type(df, recording_id):
    """Filter the DataFrame based on the recording type."""
    if recording_id == '001':
        return df.dropna(subset=['sentence', 'sentence_english'])
    elif recording_id == '002':
        return df.dropna(subset=['sentence'])
    elif recording_id == '004':
        return df.dropna(subset=['chinese', 'english'])
    elif recording_id == '005':
        return df.dropna(subset=['pinyin'])
    elif recording_id == '006':
        return df.dropna(subset=['word1', 'word1_english', 'word2', 'word2_english'])
    else:
        raise ValueError("Invalid recording ID")

# Which data to use this time?
types_not_allowed = ['phrase', 'sentence', 'part sent', 'phrase_save']
df_this = df[
        (df['priority'] <= 2) &
        (df['known_english_prompt'] >= 2) &
        (~df['type'].isin(types_not_allowed))
    ]
df_this = (filter_by_recording_type(df_this, recording_id)
    .sort_values(['category1', 'pinyin'], ascending=[True, True])
    .reset_index(drop=True))
print(df_this.shape)
df_this.head(10)

(1052, 23)


,chinese,pinyin,english,type,priority,known,known_pinyin_prompt,known_english_prompt,phonetic,category1,quality,word1,word1_english,word2,word2_english,word3,word3_english,word4,word4_english,sentence,sentence_pinyin,sentence_english,date
0,标准,biāo zhǔn,standard,combo,1.0,5.0,1.0,3.0,NaN,adjective,3.0,目标,target,准备,preparation,NaN,NaN,NaN,NaN,这家店的服务很标准,Zhè jiā diàn de fúwù hěn biāozhǔn,The service here is very standard,NaN
1,出名,chū míng,famous,combo,2.0,5.0,2.0,2.0,NaN,adjective,2.0,出,to go out,名,name,NaN,NaN,NaN,NaN,他因为短视频很快出名,Tā yīnwèi duǎn shìpín hěn kuài chūmíng,He became famous quickly because of short videos,NaN
2,大声,dà shēng,loud,combo,2.0,4.0,5.0,2.0,NaN,adjective,1.0,大,big,声,voice,NaN,NaN,NaN,NaN,请不要大声说话,Qǐng búyào dàshēng shuōhuà,Please do not speak loudly,2025-06-15
3,角度,jiǎo​ dù,angle,combo,2.0,3.0,1.0,5.0,NaN,adjective,2.0,角,horn,程度,degree,NaN,NaN,NaN,NaN,换个角度看看,Huàn gè jiǎodù kànkan,Look at it from another angle,NaN
4,平常,píng​ cháng,ordinary;common;usually,no combo,1.0,4.0,2.0,3.0,NaN,adjective,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,我平常七点起床,Wǒ píngcháng qī diǎn qǐchuáng,I usually get up at seven,NaN
5,顺利,shùn ​lì,smoothly,no combo,1.0,2.0,2.0,2.0,NaN,adjective,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,如果一切顺利我们下周就出发,Rúguǒ yīqiè shùnlì wǒmen xià zhōu jiù chūfā,If everything goes well we will leave next week,NaN
6,通常,tōng​ cháng,generally;usually,no combo,1.0,3.0,2.0,5.0,NaN,adjective,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,我通常早起,Wǒ tōngcháng zǎoqǐ,I usually wake up early,NaN
7,详细,xiáng xì,detailed,no combo,1.0,5.0,5.0,5.0,NaN,adjective,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,请给我详细地址,Qǐng gěi wǒ xiángxì dìzhǐ,Please give me the detailed address,NaN
8,正常,zhèng ​cháng,normal,no combo,1.0,3.0,2.0,3.0,NaN,adjective,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,一切都很正常,Yīqiè dōu hěn zhèngcháng,Everything is normal,NaN
9,主动,zhǔ dòng,initiative;voluntarily,combo,2.0,5.0,5.0,5.0,NaN,adjective,2.0,主意,plan,动作,movement,NaN,NaN,NaN,NaN,她主动打了招呼,Tā zhǔdòng dǎ le zhāohu,She greeted me first,NaN


In [ ]:
def pinyin_to_tones(pinyin):
    """Convert pinyin to tones."""
    pinyin = pinyin.replace('ā', 'a1').replace('á', 'a2').replace('ǎ', 'a3').replace('à', 'a4')
    pinyin = pinyin.replace('ē', 'e1').replace('é', 'e2').replace('ě', 'e3').replace('è', 'e4')
    pinyin = pinyin.replace('ī', 'i1').replace('í', 'i2').replace('ǐ', 'i3').replace('ì', 'i4')
    pinyin = pinyin.replace('ō', 'o1').replace('ó', 'o2').replace('ǒ', 'o3').replace('ò', 'o4')
    pinyin = pinyin.replace('ū', 'u1').replace('ú', 'u2').replace('ǔ', 'u3').replace('ù', 'u4')
    pinyin = pinyin.replace('ü', 'v1').replace('ǘ', 'v2').replace('ǚ', 'v3').replace('ǜ', 'v4')
    tones = []
    for pinyin_oneword in pinyin.split():
        tone = '1'  # Default tone
        for i in range(1, 5):
            if f'{i}' in pinyin_oneword:
                tone = str(i)
                break
        tones.append(tone)
    return tones

if recording_id == '005':
    df_this['pinyin_tones'] = df_this['pinyin'].apply(pinyin_to_tones)
    df_this[['pinyin', 'pinyin_tones']]

    # Make pinyin audio
    pinyin_tones = ['1', '2', '3', '4']
    for tone_str in pinyin_tones:
        gTTS(tone_str, lang='en').save(f"audio_files/english/{tone_str}.mp3")

,pinyin,pinyin_tones
0,biāo zhǔn,"[1, 3]"
1,chū míng,"[1, 2]"
2,dà shēng,"[4, 1]"
3,jiǎo​ dù,"[3, 4]"
4,píng​ cháng,"[2, 2]"
...,...,...
1047,xiū gǎi,"[1, 3]"
1048,yù bào,"[4, 4]"
1049,zhěng lǐ,"[3, 3]"
1050,zào chéng,"[4, 2]"


# 3. TTS

In [37]:
def create_tts_file(tts_type, content_str, lang_name, last_timestamp):
    if tts_type == 'zh_slow':
        slow_mode = True
    else:
        slow_mode = False

    new_file_path = f"audio_files/{tts_type}/{content_str}.mp3"
    if not os.path.exists(new_file_path):
        try:
            gTTS(content_str, lang=lang_name, slow=slow_mode).save(new_file_path)
        except:
            # Wait 60 seconds and try again
            print(f"!!!!!!! FAILURE, wait 52 seconds, row{i_row}, {tts_type}, {content_str} !!!!!!!")
            time.sleep(52)
            try:
                gTTS(content_str, lang=lang_name, slow=slow_mode).save(new_file_path)
            except:
                # Wait 60 seconds and try again
                print(f"!!!!!!!!!! FAILURE AGAIN, wait 278 seconds, row{i_row}, {tts_type}, {content_str} !!!!!!!!!!")
                time.sleep(278)
                gTTS(content_str, lang=lang_name, slow=slow_mode).save(new_file_path)
        print(f"{(time.time()-last_timestamp):.3f}s, row{i_row}, {tts_type}, {content_str}")
    else:
        print(f"{(time.time()-last_timestamp):.3f}s, ALREADY EXISTS, row{i_row}, {tts_type}, {content_str}")


for i_row, row in df_this.iterrows():
    create_tts_file(tts_type='zh', content_str=row['chinese'], lang_name='zh-cn', last_timestamp=time.time())
    create_tts_file(tts_type='zh_slow', content_str=row['chinese'], lang_name='zh-cn', last_timestamp=time.time())
    create_tts_file(tts_type='english', content_str=row['english'], lang_name='en', last_timestamp=time.time())
    # create_tts_file(tts_type='zh_tw', content_str=row['chinese'], lang_name='zh-tw', last_timestamp=time.time())
    if recording_id == '001':
        create_tts_file(tts_type='zh', content_str=row['sentence'], lang_name='zh-cn', last_timestamp=time.time())
        create_tts_file(tts_type='english', content_str=row['sentence_english'], lang_name='en', last_timestamp=time.time())
    elif recording_id == '002':
        create_tts_file(tts_type='zh', content_str=row['sentence'], lang_name='zh-cn', last_timestamp=time.time())
    elif recording_id == '006':
        create_tts_file(tts_type='zh', content_str=row['word1'], lang_name='zh-cn', last_timestamp=time.time())
        create_tts_file(tts_type='zh', content_str=row['word2'], lang_name='zh-cn', last_timestamp=time.time())
        create_tts_file(tts_type='english', content_str=row['word1_english'], lang_name='en', last_timestamp=time.time())
        create_tts_file(tts_type='english', content_str=row['word2_english'], lang_name='en', last_timestamp=time.time())
        if not pd.isna(row['word3']):
            create_tts_file(tts_type='zh', content_str=row['word3'], lang_name='zh-cn', last_timestamp=time.time())
            create_tts_file(tts_type='english', content_str=row['word3_english'], lang_name='en', last_timestamp=time.time())
        if not pd.isna(row['word4']):
            create_tts_file(tts_type='zh', content_str=row['word4'], lang_name='zh-cn', last_timestamp=time.time())
            create_tts_file(tts_type='english', content_str=row['word4_english'], lang_name='en', last_timestamp=time.time())

0.000s, ALREADY EXISTS, row0, zh, 标准
0.000s, ALREADY EXISTS, row0, zh_slow, 标准
0.000s, ALREADY EXISTS, row0, english, standard
0.000s, ALREADY EXISTS, row0, zh, 这家店的服务很标准
0.000s, ALREADY EXISTS, row0, english, The service here is very standard
0.000s, ALREADY EXISTS, row1, zh, 出名
0.000s, ALREADY EXISTS, row1, zh_slow, 出名
0.000s, ALREADY EXISTS, row1, english, famous
0.000s, ALREADY EXISTS, row1, zh, 他因为短视频很快出名
0.000s, ALREADY EXISTS, row1, english, He became famous quickly because of short videos
0.000s, ALREADY EXISTS, row2, zh, 大声
0.000s, ALREADY EXISTS, row2, zh_slow, 大声
0.000s, ALREADY EXISTS, row2, english, loud
0.000s, ALREADY EXISTS, row2, zh, 请不要大声说话
0.000s, ALREADY EXISTS, row2, english, Please do not speak loudly
0.000s, ALREADY EXISTS, row3, zh, 角度
0.000s, ALREADY EXISTS, row3, zh_slow, 角度
0.000s, ALREADY EXISTS, row3, english, angle
0.000s, ALREADY EXISTS, row3, zh, 换个角度看看
0.000s, ALREADY EXISTS, row3, english, Look at it from another angle
0.000s, ALREADY EXISTS, row4, zh,

# 4. Combine files for individual vocab words

In [ ]:
pause_100ms = AudioSegment.silent(duration=100)
pause_500ms = AudioSegment.silent(duration=500)
pause_1000ms = AudioSegment.silent(duration=1000)

for i_row, row in df_this.iterrows():
    start_time = time.time()
    new_file_path = f"audio_files/rows/{recording_id}_{row['chinese']}.mp3"
    if not os.path.exists(new_file_path):
        chinese_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['chinese']}.mp3")
        chinese_slow_audio = AudioSegment.from_mp3(f"audio_files/zh_slow/{row['chinese']}.mp3")
        english_audio = AudioSegment.from_mp3(f"audio_files/english/{row['english']}.mp3")
        
        if recording_id == '001':
            sent_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['sentence']}.mp3")
            sent_english_audio = AudioSegment.from_mp3(f"audio_files/english/{row['sentence_english']}.mp3")
            combined = chinese_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms + sent_audio + pause_500ms + sent_english_audio + pause_500ms + sent_audio + pause_1000ms

        elif recording_id == '002':
            sent_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['sentence']}.mp3")
            combined = chinese_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms + sent_audio + pause_500ms + sent_audio + pause_1000ms

        elif recording_id == '004':
            combined = chinese_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_1000ms

        elif recording_id == '005':
            tones_audio = AudioSegment.silent(duration=0)
            for pinyin_tone in row['pinyin_tones']:
                tones_audio += AudioSegment.from_mp3(f"audio_files/english/{pinyin_tone}.mp3")
                tones_audio += pause_100ms

            combined = chinese_audio + pause_500ms + tones_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_1000ms
 
        elif recording_id == '006':
            word1_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['word1']}.mp3")
            word1e_audio = AudioSegment.from_mp3(f"audio_files/english/{row['word1_english']}.mp3")
            word2_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['word2']}.mp3")
            word2e_audio = AudioSegment.from_mp3(f"audio_files/english/{row['word2_english']}.mp3")
            if not pd.isna(row['word3']):
                word3_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['word3']}.mp3")
                word3e_audio = AudioSegment.from_mp3(f"audio_files/english/{row['word3_english']}.mp3")
            if not pd.isna(row['word4']):
                word4_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['word4']}.mp3")
                word4e_audio = AudioSegment.from_mp3(f"audio_files/english/{row['word4_english']}.mp3")

            component_words_audio = word1_audio + pause_100ms + word1e_audio + pause_500ms + word2_audio + pause_100ms + word2e_audio
            if not pd.isna(row['word3']):
                component_words_audio += pause_500ms + word3_audio + pause_100ms + word3e_audio
            if not pd.isna(row['word4']):
                component_words_audio += pause_500ms + word4_audio + pause_100ms + word4e_audio
            combined = chinese_audio + pause_500ms + component_words_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms

        else:
            raise ValueError(f"Invalid recording_id: {recording_id}")

        combined.export(new_file_path, format="mp3")
        print(f"{(time.time()-start_time):.2f} seconds, row {i_row}, {row['chinese']}")
    else:
        print(f"{(time.time()-start_time):.2f} seconds, row {i_row}, {row['chinese']} ALREADY EXISTS")


0.00 seconds, row 0, 标准 ALREADY EXISTS
0.58 seconds, row 1, 出名
0.47 seconds, row 2, 大声
0.00 seconds, row 3, 角度 ALREADY EXISTS
0.00 seconds, row 4, 平常 ALREADY EXISTS
0.50 seconds, row 5, 顺利
0.00 seconds, row 6, 通常 ALREADY EXISTS
0.00 seconds, row 7, 详细 ALREADY EXISTS
0.00 seconds, row 8, 正常 ALREADY EXISTS
0.00 seconds, row 9, 主动 ALREADY EXISTS
0.00 seconds, row 10, 主观 ALREADY EXISTS
0.49 seconds, row 11, 脏
0.00 seconds, row 12, 自愿 ALREADY EXISTS
0.49 seconds, row 13, 百万
0.48 seconds, row 14, 差不多
0.46 seconds, row 15, 差点儿
0.47 seconds, row 16, 差距
0.49 seconds, row 17, 大部分
0.49 seconds, row 18, 大多数
0.48 seconds, row 19, 大概
0.00 seconds, row 20, 大小 ALREADY EXISTS
0.00 seconds, row 21, 概率 ALREADY EXISTS
0.72 seconds, row 22, 高度
0.50 seconds, row 23, 毫克
0.48 seconds, row 24, 毫米
0.48 seconds, row 25, 好几
0.00 seconds, row 26, 几十 ALREADY EXISTS
0.46 seconds, row 27, 厘米
0.46 seconds, row 28, 毛重
0.46 seconds, row 29, 难得
0.46 seconds, row 30, 频率
0.46 seconds, row 31, 平方米
0.00 seconds, row 32, 上升 A

In [ ]:
# os.remove('audio_files/zh_slow/贸易.mp3')

# 5. Combine individual vocab into final audio recording

In [ ]:
# Combine the audio files into a single file
all_audio_files = []
for i_row, row in df_this.iterrows():
    audio_file = f"audio_files/rows/{recording_id}_{row['chinese']}.mp3"
    if os.path.exists(audio_file):
        all_audio_files.append(AudioSegment.from_mp3(audio_file))

combined = all_audio_files[0]
for audio in all_audio_files[1:]:
    combined += audio
combined.export(f"audio_files/products/{recording_id}_{recording_name}.mp3", format="mp3")

<_io.BufferedRandom name='audio_files/products/004_0817_word_p2_k2_wordonly.mp3'>

# 6. 